In [31]:
from collections import deque
from dataclasses import dataclass

import numpy as np

np.set_printoptions(precision=2)


EXIT = 100
OBSTACLE = -1
EMPTY = 0

In [36]:
width = 10
height = width + 1
grid = np.zeros(shape=(height, width))

grid[height-1,width // 2] = EXIT

grid[:,:width // 2-1] = OBSTACLE
grid[:3, :] = EMPTY
sff = compute_static_field(grid, normalize=False)
print(sff)
# sff = compute_static_field(grid)
# print(sff)

[[12.66 11.66 11.24 10.83 10.41 10.   10.41 10.83 11.24 11.66]
 [12.24 11.24 10.24  9.83  9.41  9.    9.41  9.83 10.24 10.66]
 [11.83 10.83  9.83  8.83  8.41  8.    8.41  8.83  9.24  9.66]
 [  inf   inf   inf   inf  7.41  7.    7.41  7.83  8.24  8.66]
 [  inf   inf   inf   inf  6.41  6.    6.41  6.83  7.24  7.66]
 [  inf   inf   inf   inf  5.41  5.    5.41  5.83  6.24  6.66]
 [  inf   inf   inf   inf  4.41  4.    4.41  4.83  5.24  5.66]
 [  inf   inf   inf   inf  3.41  3.    3.41  3.83  4.24  5.24]
 [  inf   inf   inf   inf  2.41  2.    2.41  2.83  3.83  4.83]
 [  inf   inf   inf   inf  1.41  1.    1.41  2.41  3.41  4.41]
 [  inf   inf   inf   inf  1.    0.    1.    2.    3.    4.  ]]


In [1]:
def normalize_grid(static_field):
    return static_field / np.nanmax(static_field[static_field != np.inf])


def compute_static_field(grid, normalize=True):
    @dataclass
    class Node:
        coords: (int, int)
        obstacle: bool
        price: int = float("inf")

        def enter(self, parent):
            self.price = parent.price + self.distance(parent)

        def distance(self, other):
            sigma = np.power(self.coords[0] - other.coords[0], 2)
            sigma += np.power(self.coords[1] - other.coords[1], 2)
            return np.sqrt(sigma)

        def neighbours(self, width, height):
            valid_coords = []
            for x in [-1, 0, 1]:
                for y in [-1, 0, 1]:
                    if 0 <= self.coords[0] + x < width and 0 <= self.coords[1] + y < height:
                        if x == 0 and y == 0:
                            continue
                        valid_coords.append((self.coords[0] + x, self.coords[1] + y))
            return valid_coords

    q = deque()
    grid_nodes = []
    gate = None
    height, width = grid.shape
    for y in range(height):
        grid_nodes.append([])
        for x in range(width):
            coords = (x, y)
            np_coords = (y, x)
            node = Node(coords, grid[np_coords] < 0)
            if grid[np_coords] == 100:
                gate = Node(coords, False)
                node = gate
            grid_nodes[y].append(node)

    if not gate:
        raise ValueError("Gate is not present in the map. Can't compute static field.")

    gate.price = 0
    q.append(gate)
    while q:
        current_node = q.pop()
        while current_node.obstacle:
            current_node = q.pop()
        for coords in current_node.neighbours(width, height):
            x, y = coords
            other_node = grid_nodes[y][x]
            if not other_node.obstacle:
                distance = current_node.distance(other_node)
                if current_node.price + distance < other_node.price:
                    other_node.enter(current_node)
                    q.append(other_node)
    static_field = np.zeros(grid.shape)
    for x in range(width):
        for y in range(height):
            np_coords = (y, x)
            static_field[np_coords] = grid_nodes[y][x].price
    if normalize:
        return normalize_grid(static_field)
    return static_field
